In [1]:
import sys
sys.path.append('..')

In [2]:
from pathlib import Path

In [3]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from torchvision import models

In [38]:
size = 128

In [26]:
backbone = 'efficientnet-b0'
MODEL_FILE = f'/home/jiun/datasets/weights/grid/eff0-{size}/classifier-010000.pth'

from models.efficientnet.model import EfficientNet
model = EfficientNet.from_pretrained(backbone, num_classes=2)

Loaded pretrained weights for efficientnet-b0


In [ ]:
backbone = 'resnet101'
MODEL_FILE = '../merge-weights-resnet/classifier-000500.pth'

In [39]:
backbone = 'mobilenet_v2'
MODEL_FILE = f'/home/jiun/datasets/weights/grid/mb2-{size}/classifier-010000.pth'

In [40]:
model = getattr(models, backbone, None)()

In [41]:
model.classifier[-1] = nn.Linear(model.classifier[-1].in_features, 2)

In [42]:
model.load_state_dict(torch.load(MODEL_FILE, map_location=lambda s, l: s))
model.eval()

MobileNetV2(
  (features): Sequential(
    (0): ConvBNReLU(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): ConvBNReLU(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): ConvBNReLU(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=Tr

In [43]:
model.set_swish(memory_efficient=False)

AttributeError: 'MobileNetV2' object has no attribute 'set_swish'

In [57]:
inputs = torch.ones(1, 3, size, size)
output = model(inputs)

In [70]:
inputs = Variable(torch.FloatTensor(1, 3, size, size), requires_grad=False)
_ = model(inputs)

module = torch.jit.trace(model, inputs)
torch.jit.save(module, f'classifier-{backbone}-{size}.pt')

In [71]:
mm = torch.jit.load(f'classifier-{backbone}-{size}.pt')

In [84]:
%%time
mm.eval()
for i in range(100):
    vv = mm(inputs)

CPU times: user 5.58 s, sys: 46.9 ms, total: 5.62 s
Wall time: 937 ms


In [85]:
%%time
module.eval()
for i in range(100):
    vv = module(inputs)

CPU times: user 5.53 s, sys: 0 ns, total: 5.53 s
Wall time: 921 ms


In [86]:
%%time
model.eval()
for i in range(100):
    vv = model(inputs)

CPU times: user 6.02 s, sys: 78.1 ms, total: 6.09 s
Wall time: 1.02 s


In [ ]:
_.shape

In [ ]:
import cv2
import numpy as np

In [ ]:
from lib.augmentation import Base

In [ ]:
img = image = cv2.imread('ground.jpg')
scale = torch.Tensor([image.shape[1], image.shape[0],
                      image.shape[1], image.shape[0]])

In [ ]:
image, *_ = transform(image)
inputs = Variable(torch.from_numpy(image).permute(2, 0, 1).unsqueeze(0))

In [ ]:
output, *_ = model(inputs)

In [ ]:
print(inputs.min(), inputs.max())
print(inputs.mean(), inputs.std())

In [ ]:
print(output.min(), output.max())
print(output.mean(), output.std())

In [ ]:
confThresh = .3
nmsThresh = .3

In [ ]:
boxes.shape

In [ ]:
detection = np.empty((0, 6), dtype=np.float32)
for klass, boxes in enumerate(output):
    if not klass:
        continue
        
    candidates = boxes[boxes[:, 0] >= confThresh]

    if candidates.size(0) == 0:
        continue

    candidates[:, 1:] *= scale

    detection = np.concatenate((
        detection,
        np.hstack((
            np.full((np.size(candidates, 0), 1), klass, dtype=np.uint8),
            candidates.cpu().detach().numpy(),
        )),
    ))

In [ ]:
for klass, conf, x, y, w, h in detection:
    cv2.rectangle(img, (int(x), int(y)), (int(w), int(h)), (255, 0, 0), 3)

In [ ]:
show(img)

In [ ]:
transform = Base()

In [ ]:
IMAGE_FILE = 'ground.jpg'

In [ ]:
cp /home/jiun/datasets/amano/cylinder/refine-up-down/train/images/nvr001_10_CT_20190111015007+1.jpg tset.jpg

In [ ]:
IMAGE_FILE = '/home/jiun/datasets/amano/cylinder/refine-up-down/train/images/nvr001_10_CT_20190111015007+1.jpg'

In [ ]:
image = cv2.imread(IMAGE_FILE)

In [ ]:
scale = torch.Tensor([image.shape[1], image.shape[0],
                      image.shape[1], image.shape[0]])

In [ ]:
image, *_ = transform(image)

In [ ]:
detections, *_ = model(Variable(torch.from_numpy(image).permute(2, 0, 1).unsqueeze(0)))

In [ ]:
detection = np.empty((0, 6), dtype=np.float32)
for klass, boxes in enumerate(detections):
    candidates = boxes[boxes[:, 0] >= .3]

    if candidates.size(0) == 0:
        continue

    candidates[:, 1:] *= scale

    detection = np.concatenate((
        detection,
        np.hstack((
            np.full((np.size(candidates, 0), 1), klass, dtype=np.uint8),
            candidates.cpu().detach().numpy(),
        )),
    ))

In [ ]:
image = cv2.imread(IMAGE_FILE)

In [ ]:
for *_, x, y, w, h in detection:
    cv2.rectangle(image, (int(x), int(y)), (int(w), int(h)), (255, 0, 0), 3)
show(image)

In [ ]:
for *_, x, y, w, h in detection:
    cv2.rectangle(image, (int(x), int(y)), (int(w), int(h)), (255, 0, 0), 3)
show(image)

In [ ]:
show(img.astype(np.uint8))

In [ ]:
from PIL import Image
from IPython.display import display

def show(ary):
    display(Image.fromarray(ary))